In [13]:
import pandas as pd
from sqlalchemy import create_engine
import time
online_errors = 0
retail_errors = 0
key_errors = 0
marketing_errors = 0

In [9]:
online_db = pd.read_csv("Resources/Online.csv")
retail_db = pd.read_csv("Resources/Retail.csv")
key_db = pd.read_csv("Resources/KEY_SKU.csv")
marketing_db =pd.read_csv("Resources/Marketing_Spend.csv")
online_start_time = time.time()
retail_start_time = time.time()
key_start_time = time.time()
marketing_start_time = time.time()


In [5]:
online_db.head()

,Transaction ID,Date,Product SKU,Product,Product Category (Enhanced E-commerce),Quantity,Avg. Price,Revenue,Tax,Delivery
0,48497,20171231,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.00,34.44,19.99
1,48496,20171231,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,5.0,80.52,395.00,33.14,6.50
2,48495,20171231,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,Nest-USA,1.0,151.88,149.00,12.06,6.50
3,48494,20171231,GGOEGAEB091117,Google Zip Hoodie Black,Apparel,1.0,48.92,47.99,3.82,6.50
4,48493,20171231,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,121.30,119.00,8.31,6.50


In [8]:
retail_db.head()

,InvoiceNo,InvoiceDate,StockCode,Quantity
0,536598,2017-01-01,21421,1
1,536598,2017-01-01,21422,2
2,536598,2017-01-01,22178,26
3,536599,2017-01-01,20749,2
4,536599,2017-01-01,21056,2


In [10]:
key_db.head()

,Product SKU,StockCode
0,GGOENEBQ079099,21421
1,GGOENEBQ079199,21422
2,GGOENEBQ084699,22178
3,GGOEGAEB091117,20749
4,GGOENEBB078899,21056


In [11]:
marketing_db.head()

,Unnamed: 0,Offline Spend,Online Spend
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38
3,2017-01-04,4500,2928.55
4,2017-01-05,4500,4055.30


In [ ]:
# inner join key to retail on stock code
#then inner join new retail_db on to online_db on product sku
#then on marketing join on invoice date after changing the name

In [23]:
try:
    marketing_cols = ['InvoiceDate','Offline Budget','Online Budget']
    marketing_trans = marketing_db[marketing_cols].copy()
    
    marketing_trans = marketing_trans.rename(columns={'Unnamed:0':'InvoiceDate',
                                                      'Offline Spend':'Offline Budget',
                                                      'Online Spend':'Online Budget'})
except:    
    marketing_errors+=1


In [25]:
marketing_errors

6

In [31]:
marketing_cols = ['Unnamed: 0','Offline Spend','Online Spend']
marketing_trans = marketing_db[marketing_cols].copy()

marketing_trans = marketing_trans.rename(columns={'Unnamed: 0':'InvoiceDate',
                                                  'Offline Spend':'Offline Budget',
                                                  'Online Spend':'Online Budget'})
marketing_trans.head()

,InvoiceDate,Offline Budget,Online Budget
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38
3,2017-01-04,4500,2928.55
4,2017-01-05,4500,4055.30


Index(['Unnamed: 0', 'Offline Spend', 'Online Spend'], dtype='object')

In [32]:
connection_string = "postgres:dragon@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [33]:
engine.table_names()

['customer', 'emails', 'phone_numbers', 'courses', 'students']